In [1]:
import sys
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive')
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive/classifier')
sys.path.append('/home/SharedFolder/git/tillvolkmann/night-drive/utils')

In [2]:
import os
import cv2
import time
import glob
import shutil
import random
import subprocess
import numpy as np
from eval_utils import *

In [3]:
# specify device
device = "cuda"

# specify paths
path_to_videos = "/home/SharedFolder/CurrentDatasets/bdd100k_video_samples"
weights_classification = "/home/SharedFolder/trained_models/night-drive/weather_classifier/without_cloudy/20190323_ResNet18_without_cloudy_train_A_over/resnet18_weather_classifierbdd100k_sorted_train_A_over_epoch_39.pth"

# confidence thresholds, set between 0 and 1 to enable module
conf_thresh_classification = 0.7

# mapping dict for weather predictions
dict_weather = {
     0: "Weather: Clear",
     1: "Weather: Rainy",
     2: "Weather: Snowy",        
}

# for separating temporary folders when using multiple workers
worker_name = "worker_CAM1"

In [4]:
outfile_suffix = "_" + weights_classification.split(os.sep)[-1].split("sorted_")[-1].split("_epoch")[0]
outfile_suffix += "_C" + weights_classification.split("_epoch_")[-1].split(".pth")[0]
print(outfile_suffix)

_train_A_over_C39


In [ ]:
videos = glob.glob(path_to_videos + "/*.mov")
random.seed(123)
random.shuffle(videos)

In [ ]:
def process_video(video, dict_weather, conf_thresh_classification, device):
    # (re-) create directories for extracted frames and target video
    file_name = os.path.basename(video)
    path_name = os.path.dirname(video)
    temp_path = os.path.join(path_name, worker_name, file_name.split(".")[0])
    temp_pred_path = os.path.join(temp_path, "prediction")
    target_path = os.path.join(path_name, "demovideos_CAM")
    target_file = os.path.join(target_path,file_name.split(".mov")[0] + outfile_suffix + ".mp4")
    if not os.path.isdir(target_path):
        os.makedirs(target_path, exist_ok = True)
    elif os.path.exists(target_file):
        # do nothing if file already processed
        return 0    
    if os.path.isdir(temp_path):
        shutil.rmtree(temp_path)
    os.makedirs(temp_path, exist_ok = True)
    os.makedirs(temp_pred_path, exist_ok = True)
    # extract frames from video
    bash_cmd = ["ffmpeg", "-i", video, "-start_number", "0", "-qscale:v", "2", temp_path + "/frame-%d.jpg"]
    subprocess.call(bash_cmd)
    # process frames
    frames = glob.glob(temp_path + "/*.jpg")
    for frame in frames:
        # classify weather and get CAM
        pred_weather_class, pred_weather_score, cam_bgr = get_CAM_resnet18(frame, weights_classification, dict_weather, 3, device)
        # write weather on image
        weather_color = [(255, 255, 255) if pred_weather_score >= conf_thresh_classification else (80, 80, 80)]
        cv2.putText(cam_bgr,
                    f"{pred_weather_class} ({pred_weather_score:.2f})", 
                    (5, 715), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8, 
                    weather_color[0], 
                    2)
        # write bgr (will be transformed to rgb by open cv)
        cv2.imwrite(os.path.join(temp_pred_path, os.path.basename(frame)), 
                    cam_bgr, 
                    [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    # construct video
    bash_cmd = ["ffmpeg", "-r", "30", "-f", "image2", "-i", temp_pred_path + "/frame-%d.jpg", "-vcodec", "libx264", "-crf", "18", target_file]
    subprocess.call(bash_cmd)
    # clean-up
    shutil.rmtree(temp_path)
    return len(frames)

In [ ]:
for i in range(len(videos)):
    print(f"Processing video {i + 1} of {len(videos)}", end = "")
    tic = time.time()
    n_frames = process_video(videos[i], dict_weather, conf_thresh_classification, device)
    toc = time.time()
    if n_frames > 0:
        print(f"... done in {toc - tic:.2f}s ({((toc - tic) / n_frames):.2f}s / frame)")
    else:
        print(f"... skipped. File exists.")

Processing video 1 of 1000... skipped. File exists.
Processing video 2 of 1000... skipped. File exists.
Processing video 3 of 1000... skipped. File exists.
Processing video 4 of 1000... skipped. File exists.
Processing video 5 of 1000... skipped. File exists.
Processing video 6 of 1000... skipped. File exists.
Processing video 7 of 1000... skipped. File exists.
Processing video 8 of 1000... skipped. File exists.
Processing video 9 of 1000... skipped. File exists.
Processing video 10 of 1000... skipped. File exists.
Processing video 11 of 1000... skipped. File exists.
Processing video 12 of 1000... skipped. File exists.
Processing video 13 of 1000... skipped. File exists.
Processing video 14 of 1000... skipped. File exists.
Processing video 15 of 1000... skipped. File exists.
Processing video 16 of 1000... skipped. File exists.
Processing video 17 of 1000... skipped. File exists.
Processing video 18 of 1000... skipped. File exists.
Processing video 19 of 1000... skipped. File exists.
Pr